#Construct index of titles of the wiki files.
written by Yige Wen

----

In [0]:
from whoosh import analysis

analyzer=analysis.StandardAnalyzer(minsize=1)

In [0]:
from whoosh.index import create_in
from whoosh.fields import *

schema = Schema(title=ID(stored=True), wiki=NUMERIC(stored=True), content=TEXT(analyzer=analyzer))
ix = create_in("title_index_analyzer", schema)

prefix = "./wiki-pages-text/wiki-"
suffix  = ".txt"

In [0]:
def pre_process(title):
    tokens = title.replace("_", ' ')
    tokens = tokens.replace("-", ' ')
    tokens = tokens.replace("COLON", '')
    tokens = tokens.split()
    stack = [tokens[0]]
    
    if stack[0] == 'LRB':
        tokens = ' '.join(tokens)
        tokens = tokens.replace("LRB", '')
        tokens = tokens.replace("RRB", '')
        return tokens.split()
    
    for i in range(1, len(tokens)):
        head = stack[-1]
        if head == 'LRB':
            if tokens[i] == 'RRB':
                stack.pop()
            elif tokens[i] == 'LRB':
                stack.append(tokens[i])
            continue
        stack.append(tokens[i])
    return stack

pre_process("-LRB-I'm_Not_Your-RRB-_Steppin'_Stone")

["I'm", 'Not', 'Your', "Steppin'", 'Stone']

In [0]:
from whoosh import analysis
t = ' '.join(pre_process("-LRB-I'm_Not_Your-RRB-_Steppin'_Stone"))
print(t)
analyzer=analysis.StandardAnalyzer(minsize=1)
print([t.text for t in analyzer(t)])

I'm Not Your Steppin' Stone
['i', 'm', 'steppin', 'stone']


In [0]:
import time
begin = time.clock()

for i in range(109):
    start = time.clock()
    writer = ix.writer(procs=4, limitmb=256)
    
    if i <9:
        wikiNum = "00"+str(i+1)
    elif i<99:
        wikiNum = "0"+str(i+1)
    else:
        wikiNum = str(i+1)
    
    filename = prefix+wikiNum+suffix
    print("wiki-"+wikiNum)
    
    title = None
    for line in open(filename):
        a,_ = line.split(" ",1)
        if title == None:
                title = a
        elif title != a:
#             print('save:',title, '------>', (' '.join(pre_process(title))))
#             print('save:',title, '------>', [t.text for t in analyzer(' '.join(pre_process(title)))])
            try:
                writer.add_document(title = title, wiki = i+1, content = ' '.join(pre_process(title)))
            except:
                print(title)
            title = a
            
    writer.commit()
    print("Time used:",time.clock() - start)

print("Total Time:", time.clock() - begin)

wiki-001
Time used: 29.848207
wiki-002
--
Time used: 31.218483
wiki-003
Time used: 32.195431000000006
wiki-004
Time used: 32.710544999999996
wiki-005
Time used: 33.70732099999999
wiki-006
Time used: 33.28254000000001
wiki-007
Time used: 32.485209999999995
wiki-008
Time used: 31.959697000000006
wiki-009
Time used: 33.07505400000002
wiki-010
Time used: 33.02532100000002
wiki-011
Time used: 33.75133299999999
wiki-012
Time used: 34.555983000000026
wiki-013
Time used: 32.61891600000001
wiki-014
Time used: 32.04240699999997
wiki-015
Time used: 32.01046000000002
wiki-016
Time used: 32.02601900000002
wiki-017
Time used: 31.775590999999963
wiki-018
Time used: 32.14223900000002
wiki-019
Time used: 32.195098999999914
wiki-020
Time used: 32.41094100000009
wiki-021
Time used: 32.06257500000004
wiki-022
Time used: 32.183817999999974
wiki-023
Time used: 31.904281999999967
wiki-024
Time used: 32.08984500000008
wiki-025
Time used: 32.137077999999974
wiki-026
Time used: 32.04897600000004
wiki-027
Time u